In [22]:
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

file_deg_pre = '../data/DEGs_by_RNAseq_21dpi_wheat.txt'
file_deg_new = '../data/DEG-normalized.csv'

file_annotation = '/home/juan/Desktop/juan/bio/data/annotations/iwgsc_refseqv1.1_genes_2017July06/IWGSC_v1.1_HC_20170706.gff3'
file_fasta = "/home/juan/Desktop/juan/bio/data/genomes/iwgsc_refseqv1.0_all_chromosomes/161010_Chinese_Spring_v1.0_pseudomolecules.fasta"

In [23]:
df_gff = pd.read_csv(file_annotation, index_col=False, sep='\t', header=None, comment='#')
df_gff.columns = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']

In [24]:
fasta_index = SeqIO.index(file_fasta, 'fasta')

In [25]:
df_new = pd.read_csv(file_deg_new, sep = ',')
genes_new = df_new.gene.tolist()
df_new.head(2)

,Unnamed: 0,gene,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,21_1.treated.txt,21_1.txt,21_2.treated.txt,21_2.txt,21_3.treated.txt,21_3.txt,21_4.treated.txt,21_4.txt
0,1,TraesCS2B02G286300,319.972214,2.444493,0.224993,10.864762,1.696657e-27,5.058468e-23,77.189345,582.405920,81.420732,553.846531,93.679357,476.201883,145.899439,549.134507
1,2,TraesCS2B02G336800,2294.840884,-2.910446,0.267975,-10.860905,1.769871e-27,5.058468e-23,4973.338834,532.976635,5275.747232,513.078653,2149.477994,500.011977,3804.767645,609.328098


In [26]:
df_pre = pd.read_csv(file_deg_pre, sep = '\t')
genes_prev = df_pre.Gene.tolist() 
df_pre.head(2)

,Gene,log2FC
0,TRIAE_CS42_2BL_TGACv1_130477_AA0412140,-2.456314
1,TRIAE_CS42_7AS_TGACv1_570678_AA1839270,4.295336


In [ ]:
buffer_seqs = {}
gene_lists = {'new': genes_new,'prev': genes_prev}
for name, gene_list in gene_lists.items():
    buffer_seqs[name] = []
    cont = 0
    for gene in gene_list:
        df_extract = df_gff[(df_gff.attribute.str.contains('ID='+gene)) & (df_gff.feature=='gene')]
        gene_gff = df_extract.iloc[0]
        if gene_gff.start > gene_gff.end:
            end = gene_gff.start
            start = gene_gff.end
        else:
            end = gene_gff.end
            start = gene_gff.start
        for k,v in fasta_index.items():
            if k == gene:
                clean_seq = ''.join(str(record_dict[gene].seq).splitlines())
                new_seq = clean_seq[start:end]
                att = gene_gff.attribute
                desc = "seq:" + str(record.id)
                desc += " start:" + str(gene_gff.start)
                desc += " end:" + str(gene_gff.end)
                desc += " strand:" + str(gene_gff.strand)
                seq = SeqRecord(Seq(new_seq), id=gene, description=desc)
                buffer_seqs[name].append(seq)


In [ ]:
for name, gene_list in gene_lists.items():
    SeqIO.write(buffer_seqs[name], name + '.fasta', "fasta")

In [33]:
next(fasta_index.items())

('chr1A',
 SeqRecord(seq=Seq('CTAAACCCTAAACCCTAAACCCTAAACCCTAAACCCCTAACCCTAAACCCTAAC...TCc', SingleLetterAlphabet()), id='chr1A', name='chr1A', description='chr1A', dbxrefs=[]))